## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os
from datetime import datetime

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 1
data_dir = f"../data/Polypharmacy/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_12.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_22.pkl",'rb'))
# V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
# W1 = pkl.load(open(data_dir+"X_53.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_11_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_11_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_11_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_11_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"V1":V1,"R":R_temp_dict}

Loading data from data_dir:  ../data/Polypharmacy/sample1/


In [6]:
print("U1.shape: ",U1.shape)
print("V1.shape: ",V1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (645, 837)
V1.shape:  (837, 837)
R.shape:  (645, 645)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X2"],\
    "e2":["X2","X3"]}
    #"e6":["X4"]}

In [8]:
X_data = {
    "X1":{"1":data_dict['R'][1]["Rtrain"]},\
    "X2":{"1":U1},\
    "X3":V1}

In [9]:
X_meta = {
    "X1":["e1","e1"],\
    "X2":["e1","e2"],\
    "X3":["e2","e2"]}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[241, 208, 0], [111, 37, 1]]
Rtest_triplets2 = [[1, 159, 1], [0, 0, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  2
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
start_time = datetime.now()

In [19]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  2
num matrices:  3
num_val_sets:  1
X_val #matrices:  2
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2']
X_id:  X1
X[X_id].shape:  (645, 645)
X_id:  X2
X[X_id].shape:  (645, 837)
C_dict[e].shape:  torch.Size([645, 1482])
---
e_id:  e2
X_id_list:  ['X2', 'X3']
X_id:  X2
X[X_id].shape:  (645, 837)
X_id:  X3
X[X_id].shape:  (837, 837)
C_dict[e].shape:  torch.Size([837, 1

In [20]:
end_time = datetime.now()
runtime = end_time - start_time
runtime_seconds = runtime.total_seconds()
print(f"Total runtime = {runtime_seconds}")

Total runtime = 1.407777


#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [21]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2'])

In [22]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3'])

In [23]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [0.24374531209468842,
   0.05814984254539013,
   0.2744844853878021,
   0.05201802495867014,
   0.08959546685218811]},
 'loss_all_folds_avg_tuple': [0.24374531209468842,
  0.05814984254539013,
  0.2744844853878021,
  0.05201802495867014,
  0.08959546685218811],
 'loss_all_folds_avg_sum': 0.7179931318387389,
 'val_metric': 'auc',
 'val_perf_all_folds': {'1': {'X1': 1.0, 'X2': 0.0}},
 'val_perf_all_folds_avg': {'X1': 1.0, 'X2': 0.0},
 'val_perf_all_folds_total': {'1': 1.0},
 'val_perf_all_folds_total_avg': 1.0,
 'E': 2,
 'M': 3}

In [24]:
drug_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
protein_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()

In [25]:
import pandas as pd

In [26]:
drug_emb_df = pd.DataFrame(drug_emb)
drug_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.037469,-0.002871,-0.047453,0.021731,0.004482,0.001917,-0.003119,-0.031401,-0.002572,-0.007288,...,-0.053783,-0.002145,0.056322,-0.002455,0.007544,0.068918,0.031889,0.021579,0.071950,0.090616
1,0.028119,-0.015934,-0.036404,0.026111,0.006785,-0.013816,-0.005945,-0.028311,0.006982,0.002888,...,-0.059636,-0.010584,0.049845,-0.009985,0.020453,0.076722,0.034911,0.022328,0.079960,0.070648
2,-0.023684,0.012077,-0.023164,0.028115,-0.029406,0.036939,0.003427,-0.042475,0.025585,0.051752,...,-0.132905,0.018043,0.024735,0.012498,0.051268,0.092523,0.013372,-0.038296,0.119723,0.142418
3,0.041088,-0.018027,-0.031374,0.029885,0.014841,-0.015168,-0.011633,-0.021196,0.000080,-0.014079,...,-0.034100,-0.010318,0.060764,-0.017037,0.006775,0.063797,0.035630,0.032425,0.067154,0.068001
4,0.014265,-0.003711,-0.023553,0.025690,0.013118,0.001724,0.000833,-0.033097,0.021294,0.024326,...,-0.076035,-0.007983,0.026277,-0.012062,0.032528,0.084276,0.039575,0.007110,0.089100,0.088102


In [27]:
drug_emb_df.shape

(645, 100)

In [28]:
prot_emb_df = pd.DataFrame(protein_emb)
prot_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.005424,-0.021525,-0.056158,0.003692,0.005364,-0.040254,-0.030567,0.066402,-0.034854,-0.073128,...,0.015135,0.012548,0.033059,-0.043591,-0.003376,0.077068,-0.024012,-0.004733,0.035805,0.000194
1,-0.007639,-0.019938,-0.055303,0.003306,0.004257,-0.037856,-0.032009,0.069649,-0.032316,-0.074753,...,0.012290,0.012949,0.034559,-0.045345,-0.000204,0.087782,-0.021585,-0.004292,0.030925,0.000584
2,-0.008677,-0.018171,-0.047965,0.002450,0.001075,-0.035112,-0.029057,0.066526,-0.036876,-0.071788,...,0.012497,0.013093,0.033886,-0.042733,0.002550,0.086641,-0.020628,-0.005945,0.031092,0.000319
3,-0.003124,-0.002043,-0.060301,0.003359,0.010372,-0.044032,-0.027853,0.084707,-0.040400,-0.089960,...,0.017299,-0.003547,0.032409,-0.046702,0.002039,0.075467,-0.028733,-0.012493,0.030595,0.006038
4,0.003723,-0.000165,-0.054740,0.008519,0.006359,-0.044490,-0.032863,0.080093,-0.038024,-0.095464,...,0.016555,-0.000380,0.038892,-0.046540,0.004328,0.077670,-0.036770,-0.015933,0.032781,0.008828


In [29]:
prot_emb_df.shape

(837, 100)

In [30]:
emb_df = pd.concat([drug_emb_df, prot_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(1482, 100)

In [31]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.037469,-0.002871,-0.047453,0.021731,0.004482,0.001917,-0.003119,-0.031401,-0.002572,-0.007288,...,-0.053783,-0.002145,0.056322,-0.002455,0.007544,0.068918,0.031889,0.021579,0.071950,0.090616
1,0.028119,-0.015934,-0.036404,0.026111,0.006785,-0.013816,-0.005945,-0.028311,0.006982,0.002888,...,-0.059636,-0.010584,0.049845,-0.009985,0.020453,0.076722,0.034911,0.022328,0.079960,0.070648
2,-0.023684,0.012077,-0.023164,0.028115,-0.029406,0.036939,0.003427,-0.042475,0.025585,0.051752,...,-0.132905,0.018043,0.024735,0.012498,0.051268,0.092523,0.013372,-0.038296,0.119723,0.142418
3,0.041088,-0.018027,-0.031374,0.029885,0.014841,-0.015168,-0.011633,-0.021196,0.000080,-0.014079,...,-0.034100,-0.010318,0.060764,-0.017037,0.006775,0.063797,0.035630,0.032425,0.067154,0.068001
4,0.014265,-0.003711,-0.023553,0.025690,0.013118,0.001724,0.000833,-0.033097,0.021294,0.024326,...,-0.076035,-0.007983,0.026277,-0.012062,0.032528,0.084276,0.039575,0.007110,0.089100,0.088102


In [32]:
with open(f"emb_Polypharmacy_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    